In [1]:
import pandas as pd, numpy as np
from pathlib import Path
import fsspec
import pandas as pd

In [4]:
S3_STATIONS_TXT   = "s3://noaa-ghcn-pds/ghcnd-stations.txt"
S3_INVENTORY_TXT  = "s3://noaa-ghcn-pds/ghcnd-inventory.txt"
S3_BY_STATION     = "s3://noaa-ghcn-pds/csv/by_station/{id}.csv"
STOR = {"anon": True}

OUTDIR = Path('../data'); OUTDIR.mkdir(parents=True, exist_ok=True)
OUT_PARQUET = OUTDIR / '/home/pat6/ATMS-523-Final'
print('Output:', OUT_PARQUET.resolve())

Output: /home/pat6/ATMS-523-Final


In [5]:
colspecs = [(0,11),(12,20),(21,30),(31,37),(38,40),(41,71),(72,75),(76,79),(80,85)]
names = ['ID','LATITUDE','LONGITUDE','ELEVATION','STATE','NAME','GSN_FLAG','HCN_CRN_FLAG','WMO_ID']

stations = pd.read_fwf(S3_STATIONS_TXT, colspecs=colspecs, names=names, dtype={'ID':str,'STATE':str,'WMO_ID':str}, storage_options=STOR)
stations['NAME'] = stations['NAME'].str.strip(); stations['STATE'] = stations['STATE'].fillna('').str.strip()

inventory = pd.read_csv(
    S3_INVENTORY_TXT, sep=r'\s+', names=['ID','LAT','LON','ELEMENT','FIRSTYEAR','LASTYEAR'],
    dtype={'ID':str,'ELEMENT':str,'FIRSTYEAR':int,'LASTYEAR':int}, engine='python', storage_options=STOR
)

stations.head(), inventory.head()

(            ID  LATITUDE  LONGITUDE  ELEVATION STATE                   NAME  \
 0  ACW00011604   17.1167   -61.7833       10.1        ST JOHNS COOLIDGE FLD   
 1  ACW00011647   17.1333   -61.7833       19.2                     ST JOHNS   
 2  AE000041196   25.3330    55.5170       34.0          SHARJAH INTER. AIRP   
 3  AEM00041194   25.2550    55.3640       10.4                   DUBAI INTL   
 4  AEM00041217   24.4330    54.6510       26.8               ABU DHABI INTL   
 
   GSN_FLAG HCN_CRN_FLAG WMO_ID  
 0      NaN          NaN    NaN  
 1      NaN          NaN    NaN  
 2      GSN          NaN  41196  
 3      NaN          NaN  41194  
 4      NaN          NaN  41217  ,
             ID      LAT      LON ELEMENT  FIRSTYEAR  LASTYEAR
 0  ACW00011604  17.1167 -61.7833    TMAX       1949      1949
 1  ACW00011604  17.1167 -61.7833    TMIN       1949      1949
 2  ACW00011604  17.1167 -61.7833    PRCP       1949      1949
 3  ACW00011604  17.1167 -61.7833    SNOW       1949      194

In [6]:
fs = fsspec.filesystem('s3', **STOR)
candidates = ['USC00087205']
picked, url_map = [], {}
for sid in candidates:
    url = S3_BY_STATION.format(id=sid)
    if fs.exists(url):
        picked.append(sid); url_map[sid] = url
    if len(picked)>=4: break
print('Picked:', picked)
url_map

Picked: ['USC00087205']


{'USC00087205': 's3://noaa-ghcn-pds/csv/by_station/USC00087205.csv'}

In [9]:
def load_station_daily(url: str) -> pd.DataFrame:
    df = pd.read_csv(url, storage_options=STOR, dtype={'ID':str,'ELEMENT':str}, parse_dates=['DATE'])
    df['DATA_VALUE'] = df['DATA_VALUE'].replace(-9999, np.nan)
    wide = (df.pivot_table(index=['ID','DATE'], columns='ELEMENT', values='DATA_VALUE', aggfunc='first').reset_index())
    for c in ('TMAX','TMIN','TAVG'):
        if c in wide: wide[c] = wide[c]/10.0
    if 'PRCP' in wide: wide['PRCP'] = wide['PRCP']/10.0
    return wide.sort_values(['ID','DATE']).reset_index(drop=True)

frames = []
for sid in picked:
    w = load_station_daily(url_map[sid])
    frames.append(w); print(sid, w.shape)

daily = pd.concat(frames, ignore_index=True)
daily.head()

/tmp/ipykernel_5154/2081217567.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url, storage_options=STOR, dtype={'ID':str,'ELEMENT':str}, parse_dates=['DATE'])


USC00087205 (46166, 18)


ELEMENT,ID,DATE,DAPR,MDPR,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,WT01,WT03,WT04,WT06,WT08,WT11,WT14,WT16
0,USC00087205,1892-09-01,NaN,NaN,NaN,NaN,NaN,32.2,20.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,USC00087205,1892-09-02,NaN,NaN,NaN,NaN,NaN,31.7,20.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,USC00087205,1892-09-03,NaN,NaN,NaN,NaN,NaN,31.7,21.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,USC00087205,1892-09-04,NaN,NaN,NaN,NaN,NaN,32.2,21.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,USC00087205,1892-09-05,NaN,NaN,NaN,NaN,NaN,33.3,21.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
daily['month'] = daily['DATE'].dt.month
daily['year'] = daily['DATE'].dt.year

daily_sub = (daily[(daily['DATE'] >= '1991-01-01') & 
                   (daily['DATE'] <= '2020-12-31') &
                   (daily['month'].isin([10,11,12,1]))]).copy()
# drop na where TMIN is na
daily_sub_cln = daily_sub[daily_sub['TMIN'].notna()].copy()

### Average days per month below 32°F

In [54]:
#find number average days per month <= frost(32F Oc)
frost_df = (daily_sub_cln[daily_sub_cln['TMIN'] <=0]
           [['TMIN','year','month']].groupby(['year','month'])
           .count().reset_index()[['month','TMIN']]
           .groupby('month').mean().reset_index())

In [63]:
frost_df.rename(columns={'TMIN':'Avg_days_per_month'},inplace=True)
frost_df

ELEMENT,month,Avg_days_per_month
0,1,2.800000
1,11,1.000000
2,12,2.571429


### Average days per month below 28°F

In [59]:
#find number average days per month <= freeze(28F Oc)
freeze_df = (daily_sub_cln[daily_sub_cln['TMIN'] <=-2.2]
           [['TMIN','year','month']].groupby(['year','month'])
           .count().reset_index()[['month','TMIN']]
           .groupby('month').mean().reset_index())

In [64]:
freeze_df.rename(columns={'TMIN':'Avg_days_per_month'},inplace=True)
freeze_df

ELEMENT,month,Avg_days_per_month
0,1,1.583333
1,12,3.000000
